<a href="https://colab.research.google.com/github/Dih08/projetos-simples-e-aleatorios/blob/main/Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install keras

In [26]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU, Activation
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np



In [27]:
#imputando imagens

img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)


In [28]:
def build_generator():
    noise_shape = (100,)


    model = Sequential()
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

    model.summary()





In [29]:
def build_discriminator():

  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img, validity)


In [30]:
# Assuming these functions are defined elsewhere
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

discriminator.trainable = False


def save_imgs(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, 100))
  gen_imgs = generator.predict(noise)

def train(epochs, batch_size=128, save_interval=50):
    # Load the MNIST dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Normalize the dataset
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    intervalo_batch = int(batch_size / 2)

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], intervalo_batch)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        # Generate fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((intervalo_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs[:intervalo_batch], np.zeros((intervalo_batch, 1)))

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        z = Input(shape=(100,))
        img = generator(z)
        discriminator.trainable = False
        valid = discriminator(img)
        combined = Model(z, valid)
        combined.compile(loss='binary_crossentropy', optimizer=Adam())

        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)

        g_loss = combined.train_on_batch(noise, valid_y)

        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % save_interval == 0:
            save_imgs(epoch)

# Call the train function to start training
epochs = 10
train(epochs)



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_6 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_12 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_7 (Bat  (None, 512)               2048      
 chNormalization)                                     

In [31]:
def save_imgs(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, 100))
  gen_imgs = generator.predict(noise)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(5, 5)
  cnt = 0
  for i in range(5):
      for j in range(5):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  # Create the 'images' directory if it doesn't exist
  import os
  if not os.path.exists("images"):
      os.makedirs("images")
  fig.savefig("images/mnist_%d.png" % epoch)
  plt.close()

In [32]:
optimizer = Adam(0.0002, 0.5)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

In [33]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_9 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_19 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_10 (Ba  (None, 512)               2048      
 tchNormalization)                                    

In [34]:
z = Input(shape=(100,))
img = generator(z)

In [35]:
discriminator.trainable = False

In [36]:
valid = discriminator(img)

In [37]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [38]:
train(epochs=100, batch_size=32, save_interval=2000)

1/1 [==============================] - 0s 133ms/step
0 [D loss: 0.049782, acc.: 100.00%] [G loss: 6.769795]
1/1 [==============================] - 0s 37ms/step
1 [D loss: 0.056913, acc.: 100.00%] [G loss: 6.167862]
1/1 [==============================] - 0s 138ms/step
2 [D loss: 0.060905, acc.: 100.00%] [G loss: 5.806220]
1/1 [==============================] - 0s 121ms/step
3 [D loss: 0.062373, acc.: 100.00%] [G loss: 5.758904]
1/1 [==============================] - 0s 22ms/step
4 [D loss: 0.061767, acc.: 100.00%] [G loss: 5.335097]
1/1 [==============================] - 0s 22ms/step
5 [D loss: 0.062284, acc.: 100.00%] [G loss: 5.020932]
1/1 [==============================] - 0s 27ms/step
6 [D loss: 0.071349, acc.: 100.00%] [G loss: 4.765901]
1/1 [==============================] - 0s 24ms/step
7 [D loss: 0.076617, acc.: 100.00%] [G loss: 4.494195]
1/1 [==============================] - 0s 28ms/step
8 [D loss: 0.102510, acc.: 93.75%] [G loss: 4.500502]
1/1 [==============================

In [39]:
generator.save('generator.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
